# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [33]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import requests
import time
import csv
import random
import seaborn as sns
import urllib
from citipy import citipy
from scipy.stats import linregress
from urllib.error import HTTPError
from api_keys import api_key
import requests
from pprint import pprint

# Import API key
from api_keys import g_key
from api_keys import api_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [34]:
# Load data from CSV to DataFrame
cityWeather = pd.read_csv('output_data/cities.csv', index_col=False)

# Rename index
cityWeather = cityWeather.set_index('Unnamed: 0')
cityWeather = cityWeather.rename_axis('index')

# Check quantity of data loaded
len(cityWeather)


KeyError: "None of ['Unnamed: 0'] are in the columns"

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:

# Configure maps
gmaps.configure(api_key=g_key)

In [26]:
# Store lat and long for locations and humidity for weight
locations = cityWeather[['Lat', 'Lng']].astype(float)
weight = cityWeather['Humidity'].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = weight, dissipating=False, max_intensity=100, point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
# Drop rows with NN vale
cityWeather = cityWeather.dropna()

# Ideal weather conditions to narrow down data
cityWeather2 = cityWeather[cityWeather['Max Temp'] <= 290]
cityWeather2 = cityWeather2[cityWeather2['Humidity'] < 75]
cityWeather2 = cityWeather2[cityWeather2['Wind Speed'] < 4]
cityWeather2 = cityWeather2[cityWeather2['Cloudiness'] < 10]
cityWeather2

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
31,31,qingdao,0,CN,1558378760,54,36.09,120.35,59.00,2.24
142,142,sarai mir,0,IN,1558378781,35,26.02,82.92,86.00,3.36
149,149,ganzhou,1,CN,1558378783,66,25.86,114.93,64.32,3.42
167,167,yumen,2,CN,1558378785,10,40.29,97.04,59.10,1.50
205,205,hutang,4,CN,1558378793,71,29.22,119.85,59.10,3.31
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
346,346,chiredzi,0,ZW,1558378856,42,-21.05,31.67,65.76,0.78
494,494,baylo,1,PH,1558378889,69,7.85,126.06,77.00,1.07
500,500,mandalgovi,0,MN,1558378890,36,45.76,106.27,48.66,1.34
525,525,mecca,1,SA,1558378872,48,21.43,39.83,88.62,1.74


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [28]:
# Creating hotel_df with narrowed down data
hotel_df = cityWeather2

# Add columns to store hotel data
hotel_df['Hotel Name'] = ""
hotel_df['Hotel Lat'] = ""
hotel_df['Hotel Lon'] = ""
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Hotel Lat,Hotel Lon
31,31,qingdao,0,CN,1558378760,54,36.09,120.35,59.00,2.24,,,
142,142,sarai mir,0,IN,1558378781,35,26.02,82.92,86.00,3.36,,,
149,149,ganzhou,1,CN,1558378783,66,25.86,114.93,64.32,3.42,,,
167,167,yumen,2,CN,1558378785,10,40.29,97.04,59.10,1.50,,,
205,205,hutang,4,CN,1558378793,71,29.22,119.85,59.10,3.31,,,
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,,,
346,346,chiredzi,0,ZW,1558378856,42,-21.05,31.67,65.76,0.78,,,
494,494,baylo,1,PH,1558378889,69,7.85,126.06,77.00,1.07,,,
500,500,mandalgovi,0,MN,1558378890,36,45.76,106.27,48.66,1.34,,,
525,525,mecca,1,SA,1558378872,48,21.43,39.83,88.62,1.74,,,


In [35]:
# params dictionary for each iteration to retrieve data
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}

# For loop will identify nearby hotels using lat/lng of narrowed down cities
for column, row in hotel_df.iterrows():
    print(f"Retrieving Hotel Data for {row['City']}, {row['Country']}.")
    
    # Retrieve lat and lng from dataframe
    lat = row['Lat']
    lng = row['Lng']
    
    # Uses new location lat and lng during each iteration
    params["location"] = f"{lat},{lng}"
    
    # Make API call to retrieve data from Google Maps API
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address = requests.get(base_url, params=params).json()
    
    # Checks for missing data using try-except and loads in NaN for any missing data
    try:
        print(f"Closest Hotel is {name_address['results'][0]['name']}")
        
        hotel_df.loc[column, 'Hotel Name'] = name_address['results'][0]['name']
        hotel_df.loc[column, 'Hotel Lat'] = name_address['results'][0]['geometry']['location']['lat']
        hotel_df.loc[column, 'Hotel Lon'] = name_address['results'][0]['geometry']['location']['lng']
    except (KeyError, IndexError):
        print("No hotel found within radius. Skipping...")
        hotel_df.loc[column, 'Hotel Name'] = np.NaN
        hotel_df.loc[column, 'Hotel Lat'] = np.NaN
        hotel_df.loc[column, 'Hotel Lon'] = np.NaN
        
    print(f"Data processing for {row['City']}, {row['Country']} is complete.")
    print("-----------------------")

hotel_df

Retrieving Hotel Data for qingdao, CN.
No hotel found within radius. Skipping...
Data processing for qingdao, CN is complete.
-----------------------
Retrieving Hotel Data for sarai mir, IN.
No hotel found within radius. Skipping...
Data processing for sarai mir, IN is complete.
-----------------------
Retrieving Hotel Data for ganzhou, CN.
No hotel found within radius. Skipping...
Data processing for ganzhou, CN is complete.
-----------------------
Retrieving Hotel Data for yumen, CN.
No hotel found within radius. Skipping...
Data processing for yumen, CN is complete.
-----------------------
Retrieving Hotel Data for hutang, CN.
No hotel found within radius. Skipping...
Data processing for hutang, CN is complete.
-----------------------
Retrieving Hotel Data for mogok, MM.
No hotel found within radius. Skipping...
Data processing for mogok, MM is complete.
-----------------------
Retrieving Hotel Data for chiredzi, ZW.
No hotel found within radius. Skipping...
Data processing for chir

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Hotel Lat,Hotel Lon
31,31,qingdao,0,CN,1558378760,54,36.09,120.35,59.00,2.24,NaN,NaN,NaN
142,142,sarai mir,0,IN,1558378781,35,26.02,82.92,86.00,3.36,NaN,NaN,NaN
149,149,ganzhou,1,CN,1558378783,66,25.86,114.93,64.32,3.42,NaN,NaN,NaN
167,167,yumen,2,CN,1558378785,10,40.29,97.04,59.10,1.50,NaN,NaN,NaN
205,205,hutang,4,CN,1558378793,71,29.22,119.85,59.10,3.31,NaN,NaN,NaN
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,NaN,NaN,NaN
346,346,chiredzi,0,ZW,1558378856,42,-21.05,31.67,65.76,0.78,NaN,NaN,NaN
494,494,baylo,1,PH,1558378889,69,7.85,126.06,77.00,1.07,NaN,NaN,NaN
500,500,mandalgovi,0,MN,1558378890,36,45.76,106.27,48.66,1.34,NaN,NaN,NaN
525,525,mecca,1,SA,1558378872,48,21.43,39.83,88.62,1.74,NaN,NaN,NaN


In [36]:
# Drops rows with NaN value
hotel_df = hotel_df.dropna()
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Hotel Lat,Hotel Lon


In [37]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """

Name
{Hotel Name}

City
{City}
Country
{Country}

"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Hotel Lat", "Hotel Lon"]]

In [38]:
weight = hotel_df["Humidity"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
heat_layer = gmaps.heatmap_layer(locations, weights = weight, dissipating=False, max_intensity=100, point_radius=1)

# Add layer
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig

TraitError: The 'locations' trait of a WeightedHeatmap instance must be of length 1 <= L <= 9223372036854775807, but a value of [] was specified.